In [ ]:
# import musicbrainsngs as mbz
# import lyricsgenius as lg
import pandas as pd
import os, requests

In [20]:
# Establishing API keys
LASTFM_API_KEY = "d56234450cf7ebb02233e9ce6af09a7f"

In [48]:
# Gathering top tracks of desired genres
BASE_URL = "http://ws.audioscrobbler.com/2.0/"
genres = ["pop", "rock", "electronic", "country",
          "religious", "metal", "hip hop", "jazz",
          "R&B", "classical"] # Derived from musicmap.info
raw_data = {}


def getTopTracks(genre, limit=10, key=0):    
    params = {
        "method": "tag.gettoptracks",
        "tag": genre,
        "api_key": LASTFM_API_KEY,
        "format": "json",
        "limit": limit
    }
    
    response = requests.get(BASE_URL, params=params)
    data = response.json()

    # Parsing results
    for track in data["tracks"]["track"]:
        raw_data[key] = {
            "name": track["name"],
            "artist": track["artist"]["name"],
            "genre": genre,
            "duration": track["duration"]
        }
        key += 1
    return key

In [49]:
key = 0
for genre in genres:
    key = getTopTracks(genre, 2, key)

raw_data

{0: {'name': 'Pink Pony Club',
  'artist': 'Chappell Roan',
  'genre': 'pop',
  'duration': '259'},
 1: {'name': 'no tears left to cry',
  'artist': 'Ariana Grande',
  'genre': 'pop',
  'duration': '205'},
 2: {'name': 'Sparks',
  'artist': 'Coldplay',
  'genre': 'rock',
  'duration': '227'},
 3: {'name': 'Iris',
  'artist': 'Goo Goo Dolls',
  'genre': 'rock',
  'duration': '290'},
 4: {'name': 'On Melancholy Hill',
  'artist': 'Gorillaz',
  'genre': 'electronic',
  'duration': '233'},
 5: {'name': 'Feel Good Inc.',
  'artist': 'Gorillaz',
  'genre': 'electronic',
  'duration': '222'},
 6: {'name': "TEXAS HOLD 'EM",
  'artist': 'Beyoncé',
  'genre': 'country',
  'duration': '233'},
 7: {'name': '16 CARRIAGES',
  'artist': 'Beyoncé',
  'genre': 'country',
  'duration': '227'},
 8: {'name': 'Judah Smith Interlude',
  'artist': 'Lana Del Rey',
  'genre': 'religious',
  'duration': '276'},
 9: {'name': 'Ave Maria',
  'artist': 'Beyoncé',
  'genre': 'religious',
  'duration': '221'},
 10: {